In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import AutoModelForCausalLMWithValueHead

model_name = "apple/OpenELM-270M-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-hf")

c:\Users\jorin\.conda\envs\AGI\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
A new version of the following files was downloaded from https://huggingface.co/apple/OpenELM-270M-Instruct:
- configuration_openelm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/apple/OpenELM-270M-Instruct:
- modeling_openelm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
c:\Users\jorin\.conda\envs\AGI\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses syml

In [4]:
# add hidden_size attribute to the model config
# take value from model_dim attribute in config
model.config.hidden_size = model.config.model_dim

In [5]:
# push to hub
new_name = "jeggers/OpenELM-270M-Instruct"
model.push_to_hub(new_name)
tokenizer.push_to_hub(new_name)

model.safetensors: 100%|██████████| 1.09G/1.09G [12:13<00:00, 1.48MB/s]   
c:\Users\jorin\.conda\envs\AGI\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jorin\.cache\huggingface\hub\models--jeggers--OpenELM-270M-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
tokenizer.model: 100%|███████

CommitInfo(commit_url='https://huggingface.co/jeggers/OpenELM-270M-Instruct/commit/1d0c39b78a165d32275707622fcd2fef721ce8e8', commit_message='Upload tokenizer', commit_description='', oid='1d0c39b78a165d32275707622fcd2fef721ce8e8', pr_url=None, pr_revision=None, pr_num=None)

In [7]:
# try to load new model with trl

trl_model = AutoModelForCausalLMWithValueHead.from_pretrained(new_name, trust_remote_code=True)

In [18]:
# generate some text
input_texts = ["How to make a cake", "How to fix a car", "How to build a house"]
batch = tokenizer(input_texts, padding="longest", return_tensors="pt")
print(batch)

{'input_ids': tensor([[   1, 1128,  304, 1207,  263,  274, 1296],
        [   0,    1, 1128,  304, 2329,  263, 1559],
        [   0,    1, 1128,  304, 2048,  263, 3699]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1],
        [0, 1, 1, 1, 1, 1, 1],
        [0, 1, 1, 1, 1, 1, 1]])}


In [20]:

out = trl_model.generate(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"], max_new_tokens=20, do_sample=True, temperature=1.0)
out_texts = tokenizer.batch_decode(out, skip_special_tokens=True)
print(out_texts)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


["How to make a cake base...but not overly complicated.\n\nLooking around, I've found tutorials about", "How to fix a car with no keys\n\nSo I'm currently in a rented car driving up to a friends", 'How to build a house\nby Matt Toth, May 22, 2017\n"I cannot']


In [21]:
tokenizer = AutoTokenizer.from_pretrained("facebook/galactica-125m")

c:\Users\jorin\.conda\envs\AGI\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [34]:
tokenizer.clean_up_tokenization_spaces

True

In [38]:
text = "This thing costs £4.56</s>"
toks = tokenizer(text, return_tensors="pt")
print(len(toks["input_ids"][0]))
# decode and encode again
text = tokenizer.decode(toks["input_ids"][0], skip_special_tokens=True)
print(text)
toks = tokenizer(text, return_tensors="pt")
print(len(toks["input_ids"][0]))


9
This thing costs £4.56
8
